In [202]:
import sys
import numpy as np
import pandas as pd
import PIL
import collections
import heapq
from typing import Tuple, Iterable, Set
print(sys.version)
print(np.__version__)
print(pd.__version__)
print(PIL.__version__)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
1.25.2
2.1.0
10.0.0


In [203]:
from Map import Map_Obj

In [204]:
task1map = Map_Obj(task=1)
task2map = Map_Obj(task=2)
task3map = Map_Obj(task=3)
task4map = Map_Obj(task=4)
task5map = Map_Obj(task=5)

In [205]:
#SquareGrid class
#takes in a map object and creates a grid of the map
#map.int_map is a numpy array of the map with -1 as walls and 1 to 4 is the different cost of terrain
class SquareGrid:
    def __init__(self, map):
        self.map = map.int_map
        self.width = map.int_map.shape[1]
        self.height = map.int_map.shape[0]
        self.weights: dict[list[int, int], int] = {}

    def in_bounds(self, id: list[int, int]) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    def passable(self, id: list[int, int]) -> bool:
        return self.map[id[0], id[1]] != -1

    def neighbors(self, id: list[int, int]) -> Iterable[Tuple[int, int]]:
        (x, y) = id
        neighbors = [(x + 1, y), (x - 1, y), (x, y - 1), (x, y + 1)]
        if (x + y) % 2 == 0:
            neighbors.reverse()
        neighbors = filter(self.in_bounds, neighbors)
        neighbors = filter(self.passable, neighbors)
        return neighbors
    
    def cost(self, from_node: list[int, int], to_node: list[int, int]) -> int:
        return self.map[to_node[0], to_node[1]]

In [206]:
SquareGrid = SquareGrid(task3map)
id=(7,30)
print(SquareGrid.width, SquareGrid.height)
print(SquareGrid.in_bounds(id))
print(SquareGrid.passable(id))
print(task1map.get_cell_value(id))
neighbors = list(SquareGrid.neighbors(id))
print(neighbors)
for best in neighbors:
    print(SquareGrid.cost(id, best))

39 47
True
True
1
[(8, 30), (6, 30), (7, 31)]
2
1
1


In [207]:
#priority queue
class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[int, list[int,int]]] =[]
        
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: list[int,int], priority: int):
        heapq.heappush(self.elements, (priority, item))
        
    def get(self) -> list[int,int]:
        return heapq.heappop(self.elements)[1]

In [208]:
def heuristic(a: list, b: list)-> float:
    """
    Heuristic function h

    parameters
    ---------
    a: list 
        current position
    b: list
        goal position
    returns
    -------
    float
        heuristic value
        manhattan distance from current position to goal position
    """
    x1, y1 = a[0], a[1]
    x2, y2 = b[0], b[1]
    distance=abs(x1 - x2) + abs(y1 - y2)
    #print(distance)
    return distance

In [209]:
#implementing A* algorithm 

def A_star(start: list, goal: list, map_obj: Map_Obj)-> list:
    """
    A* algorithm implementation 

    parameters
    ----------
    start: list
        start position
    goal: list
        goal position
    map_obj: Map_Obj
        map object
        
    returns
    -------
    list
        path from start to goal

    """
    #initializing open and closed list
    open_list = []
    closed_list = []
    #adding start node to open list
    open_list.append(start)
    #looping until open list is empty
    while open_list:
        #getting current node
        current_node = open_list[0]
        current_index = 0
        #looping through open list to find node with lowest f value
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
        #removing current node from open list and adding it to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)
        #checking if goal is reached
        if current_node == goal:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1]
        #getting current node's neighbors
        neighbors = map_obj.get_neighbors(current_node.position)
        #looping through neighbors
        for neighbor in neighbors:
            #checking if neighbor is in closed list
            if neighbor in closed_list:
                continue
            #calculating g and h values
            neighbor.g = current_node.g + 1
            neighbor.h = heuristic(neighbor.position, goal.position)
            neighbor.f = neighbor.g + neighbor.h
            #checking if neighbor is in open list
            if neighbor in open_list:
                continue
            #adding neighbor to open list
            open_list.append(neighbor)



In [210]:
#task3map.show_map()

In [211]:
task3map.get_maps()

(array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1],
    